# RNAge Progress Report

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML,Image
from pathlib import Path
data_dir=Path("cs418-project-RNAge/data") # change
tissue_dir=Path("tissue-specific")
#!mkdir data && cp merged_meta.tsv data #Needed after cloning repo
manifest={"data":"All_Tissue_Site_Details.combined.reads.gct",
              "sample_meta":"GTEx_v7_Annotations_SampleAttributesDS.txt",
              "subject_meta":"GTEx_v7_Annotations_SubjectPhenotypesDS.txt",
               "merged_meta":"merged_meta.tsv"}

meta=pd.read_csv(data_dir/manifest['merged_meta'],sep="\t",dtype={'SMUBRID':object,'SEX':object,'DTHHRDY':object})

## Introduction

Text

## Changes

Text

## Data Cleaning

Text
Include mapping subject attributes to sample attributes with dataclass.py.

Include removing samples without age.

Include count and variance filtering.

## EDA

Our first EDA task was to understand how many samples were from each tissue origin. This table shows how many samples are from each tissue. Interestingly, some of the lowest count tissues are sex organs. Some of the most collected tissues seem that they may have been easier to collect.

In [7]:
counts=pd.DataFrame(meta['SMTS'].value_counts())
display(counts)

,SMTS
Brain,1671
Skin,1203
Esophagus,1021
Blood Vessel,913
Adipose Tissue,797
Heart,600
Muscle,564
Blood,537
Colon,507
Thyroid,446


We also found that many samples do not have ages recorded.
![](progress_plots/AGE_BAR.png)

AGE,20-29,30-39,40-49,50-59,60-69,70-79
SMTS,,,,,,
Adipose Tissue,57,66,131,273,245,25
Blood,50,46,103,169,163,6
Blood Vessel,75,75,160,310,273,20
Brain,12,8,36,112,152,11
Breast,26,32,53,88,81,10
Colon,45,48,95,164,140,15
Esophagus,102,91,191,356,258,23
Heart,33,29,95,220,205,18
Lung,27,30,76,145,139,10


Tissues had various sub-tissue locations. Only some tissues had sub-tissue location recorded, but those who did appeared to display grouping along that basis in PCA plots.
![](progress_plots/Blood_Vessel_SMTSD_PCA.png)
![](progress_plots/Brain_SMTSD_PCA.png)

We found that library sizes (the total number of reads sequenced and aligned into counts) were generally around 100 million, which is good, but there are some outliers for each tissue. This plot shows the distribution of library sizes for Colon samples.
![](progress_plots/libsizeplot.png)

## Visualization

We knew that some tissues had very few samples. To begin testing the relationships of the data, we needed to establish a threshold of minimum number of samples per tissue. This table shows the number of 

In [28]:
df=meta[meta['SMTS'].isin(counts[counts['SMTS']>1].index)]
df=pd.crosstab(index=df['SMTS'],columns=df['AGE'])
sorted_i=df.sum(axis=1).sort_values(ascending=False).index.values
df=df.loc[sorted_i,:]
display(df)

AGE,20-29,30-39,40-49,50-59,60-69,70-79
SMTS,,,,,,
Skin,98,94,200,398,377,35
Esophagus,102,91,191,356,258,23
Blood Vessel,75,75,160,310,273,20
Adipose Tissue,57,66,131,273,245,25
Heart,33,29,95,220,205,18
Muscle,46,45,88,188,179,18
Blood,50,46,103,169,163,6
Colon,45,48,95,164,140,15
Thyroid,30,29,81,151,143,12


## ML

Text

## Reflection

- The size of the data has been extremely prohibitive. The transformed counts are floats, which take much more space. It is best to compute the cpm transformation on the entire data set, meaning that we had to create these floats all in R before moving to Python.

## Next Steps

Text